# TensorFlow Hub 를 이용한 Transfer Learning

- Pre-trained Moblenet 사용  
- GPU 필요하므로 Colab에서 작업

### Tensorflow Hub 에서 PRE-TRAINED MOBILENET 의 WEIGHT 를 가져옴  

TensorFlow Hub에서 제공하는 MobileNet_v2 사전 훈련 모델을 사용하여 입력 형태가 (224, 224, 3)인 단일 계층으로 구성된 Keras 모델을 생성합니다. 이 모델은 주로 이미지 분류 작업에 사용되는 MobileNet_v2 아키텍처를 기반으로 합니다.

Trained_Mobilenet.input: 이것은 모델의 입력 텐서입니다. 모델의 입력 형태는 (224, 224, 3)이며, 이는 RGB 이미지의 높이, 너비, 채널 수를 나타냅니다.  
Trained_Mobilenet.output: 이것은 모델의 출력 텐서입니다. 출력은 모델이 이미지를 분류하기 위해 사용하는 분류 점수를 나타냅니다. MobileNet_v2는 기본적으로 ImageNet 데이터 세트에 대해 훈련되어 있으므로, 출력 벡터의 길이는 1001이 됩니다. 각 요소는 해당 클래스의 확률 점수를 나타냅니다.

### PRE-TRAINED MOBILENET 평가

URL에서 이미지를 다운로드합니다. 이 작업은 request.urlopen() 함수를 사용하여 수행되며, .read() 함수로 이미지 데이터를 바이트로 읽습니다.  
다운로드한 이미지를 PIL 라이브러리를 사용하여 224x224 크기로 리사이즈합니다.  
BytesIO는 파이썬의 io 모듈에 포함된 클래스로, 바이트 스트림을 메모리 상에서 파일처럼 다룰 수 있게 해줍니다. 이를 사용하면 디스크에 파일을 저장하지 않고도 바이트 데이터를 파일 객체처럼 읽고 쓸 수 있습니다.  
BytesIO 객체를 사용하여 이미지 데이터를 PIL 이미지 객체로 변환하고, .resize() 함수를 사용하여 이미지를 (224, 224) 크기로 리사이즈합니다.

tf.keras.applications.mobilenet.preprocess_input 함수는 이미지 데이터를 MobileNet 모델에 맞게 전처리하는 역할을 합니다. 이 함수는 주어진 이미지를 NumPy 배열로 변환하고, 모델에 사용되는 특정 범위로 픽셀 값을 조정합니다 (예: -1에서 1).

Sample_Image는 PIL.Image 객체로 크기가 (224, 224)로 조정되어 있습니다. 이 이미지를 np.array()를 사용하여 NumPy 배열로 변환한 다음, preprocess_input 함수를 사용하여 전처리를 수행합니다.

x의 차원 (224, 224, 3)는 전처리된 이미지가 224x224 크기의 컬러 이미지임을 나타냅니다. 각 숫자는 각각 높이, 너비, 색상 채널 (RGB)을 나타냅니다.

np.expand_dims(x, axis=0)를 사용하여 이미지 배열의 모양을 (1, 224, 224, 3)으로 변경합니다. 이렇게 하면 입력 이미지가 하나의 배치로 처리되며, 모델은 한 번에 하나의 이미지를 예측할 수 있습니다.

decode_predictions 함수는 모델이 예측한 결과를 사람이 이해하기 쉬운 형태로 변환합니다. 이 함수는 예측된 클래스의 확률을 기반으로 상위 K개의 예측 결과를 반환합니다.  
predicted_class[:, 1:] 코드는 첫 번째 레이블(배경)을 제외한 나머지 확률 값을 추출합니다. 이렇게 하면 배경 레이블을 고려하지 않고 상위 예측 결과를 얻을 수 있습니다.

TensorFlow에서 제공하는 get_file 함수를 사용하여 'ImageNetLabels.txt' 파일을 다운로드합니다. 이 파일에는 ImageNet 데이터셋에 포함된 레이블들이 저장되어 있습니다. 다운로드된 파일은 labels_path 변수에 저장됩니다.  
open(labels_path).read().splitlines() 코드를 사용하여 다운로드한 텍스트 파일을 읽고 각 줄을 나눠 imagenet_labels라는 NumPy 배열에 저장합니다. 

 np.argmax(predicted_class)를 사용하여 예측된 클래스 확률 중 가장 큰 값을 찾습니다.

###  Fine Tuning 전 특정 domain 의 Batch Image 에 대한 MobileNet 평가 

MobileNet 은 Flower 에 특화된 model 이 아니므로 정확도 낮을 것으로 예상됩니다.  
get_file 함수를 사용하여 인터넷에서 꽃 이미지 데이터셋을 다운로드합니다.    다운로드할 파일의 이름을 'flower_photos'로 지정하고, 데이터셋의 URL을 인자로 전달합니다. untar=True는 파일이 tar 압축 형식이므로 다운로드 후 압축을 풀어야 함을 나타냅니다.

TensorFlow의 ImageDataGenerator를 사용하여 꽃 이미지 데이터셋에 대한 데이터 전처리를 수행합니다. ImageDataGenerator는 실시간 데이터 전처리 및 증강을 수행하며, 이미지 파일을 학습 데이터로 변환하는 데 사용됩니다. 이 예제에서는 MobileNet의 전처리 함수를 사용하여 이미지를 전처리합니다.  
ImageDataGenerator 객체인 image_generator를 생성한 후, flow_from_directory 함수를 사용하여 로컬 파일 시스템에서 이미지를 로드하고 전처리합니다. 이 함수는 이미지를 로드하고, 지정된 target_size(224 x 224)로 크기를 조정하며, 배치 크기를 64로 설정하여 데이터를 배치 단위로 반환합니다. shuffle=True는 데이터를 무작위로 섞어 과적합을 방지합니다.


next() 함수를 사용하여 flowers_data에서 배치를 추출합니다.  
input_batch는 전처리된 이미지 데이터의 배치입니다. 이 경우, 배치 크기는 64이며, 각 이미지의 크기는 224 x 224 x 3이므로 배치의 모양(shape)은 (64, 224, 224, 3)입니다.  
label_batch는 이미지에 해당하는 원-핫 인코딩된 레이블 데이터의 배치입니다. 이 경우, 클래스 수가 5개이므로 레이블 배치의 모양은 (64, 5)입니다.  
flowers_data.num_classes는 데이터셋의 클래스 수를 반환합니다. 이 경우, 꽃 데이터셋에는 5개의 클래스가 있습니다.  
flowers_data.class_indices는 클래스 이름과 해당 인덱스를 포함하는 딕셔너리입니다. 이 정보는 이미지의 실제 레이블과 예측된 레이블을 비교할 때 유용합니다.

 flowers_data.class_indices 딕셔너리의 키와 값의 위치를 바꾸어 새로운 딕셔너리 class_names를 생성합니다. 이렇게 하면, 인덱스를 사용하여 각 클래스의 이름을 쉽게 찾을 수 있습니다.

첫 번째 배치에서 10개의 이미지를 시각화하여 그리드에 표시 합니다. 각 이미지의 전처리를 되돌려 정규화된 값을 원래 이미지의 값으로 변환합니다.

임의의 꽃 image 1 개를 선택하여  예측을 수행합니다.


decode_predictions() 함수는 1000개의 클래스 레이블 중 확률이 가장 높은 상위 5개 결과를 인간이 이해할 수 있는 레이블로 변환해주는 함수입니다.  이 레이블은 [(class_id, class_name, class_probability)] 형태의 튜플 리스트로 반환합니다.


### 전이학습 MODEL 을 Flower 분류에 적합한 model 로 Retrain 

### Fine Tuning 을 위해 head 가 제거된 model 을 download  

- https://tfhub.dev/s?module-type=image-feature-vector&q=tf2

extractor_layer를 통해 입력 이미지 배치를 모바일넷으로 처리한 특성 벡터 배치를 얻습니다. 반환된 feature_batch의 shape를 출력해보면면, 64개의 이미지에 대해, 각 이미지는 1280 차원의 특성 벡터로 나타납니다.

extractor_layer.trainable = False는 모델의 특정 레이어를 학습에서 제외시키는 역할을 합니다. 이를 통해 레이어의 가중치가 업데이트되지 않도록 하여 해당 레이어의 특성을 고정시켜 놓을 수 있습니다. 

### Model을 두 부분으로 구성:
-    (1)  MobileNet Feature Extractor 
-   (2)  출력 부분에 Dense Network (classifier) 추가 

- output shape 이 정확한지 training 전에 사전 check해 봅니다.

### Flower 분류 전문으로 Fine Tuning 된 MODEL 평가

np.argmax() 함수를 이용하여 각 샘플의 예측값 중 가장 큰 값(즉, 가장 높은 확률값)에 해당하는 인덱스를 y_pred에 저장합니다.

 y_pred와 y_true 간의 일치하는 값의 비율을 계산하여 백분율로 나타냅니다

이전과 마찬가지로  각 이미지의 전처리를 되돌려 정규화된 값을 원래 이미지의 값으로 변환합니다.